In [ ]:
from cross_validation import *
from losses import *
from preprocessing import *
from metrics import *
from helpers import *
from submissions import *
from Networks import *

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu' 
torch.manual_seed(42) # Make the training reproducible as much as possible
np.random.seed(0)

In [ ]:
Images, GroundTruths = get_sets()

In [ ]:
Images, GroundTruths = preprocessing(Images, GroundTruths, size = 320)
train_loader, validation_loader, test_loader = make_dataloaders(Images, GroundTruths, 0.8, 0.1, 0.1)

In [ ]:
net = UNet(0.5).to(device)
criterion = BCE_Dice_Loss()
optimizer = optim.Adam(net.parameters(), lr = 2e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.5)

train(net, train_loader, validation_loader, criterion, optimizer, scheduler, 100, 'Best_Net', device, use_scheduler = True)

In [ ]:
test(net, validation_loader, criterion, device)

In [ ]:
net_name = 'Best_Net'
net = net = UNet(0.5).to(device)
net.load_state_dict(torch.load(net_name)['state'])
net.metrics = torch.load(net_name)['metrics']

In [ ]:
create_submission(net, 'Best_Submission.csv', device)